In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np
import re
from mlxtend.frequent_patterns import apriori, association_rules
import en_core_web_sm
spc_en = en_core_web_sm.load()

In [3]:
course = pd.read_csv("archive/course.csv")

In [4]:
course = course.drop(['Unnamed: 0', 'Course_id'], axis=1)

In [5]:
course_ids = [str(i).zfill(4) for i in range(1, len(course) + 1)]

# Thêm cột "Course_id" vào DataFrame
course.insert(0, 'Course_id', course_ids)

In [ ]:
course

In [7]:
history = pd.read_csv("history.csv", encoding='latin-1')
history
data = list(history['History_course_id'].apply(lambda x:x.split(",") ))

In [8]:
#Let's transform the list, with one-hot encoding
from mlxtend.preprocessing import TransactionEncoder
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
df = pd.DataFrame(a_data,columns=a.columns_)
df = df.replace(False,0)
df

,0005,0005,0014,0022,0024,0031,0044,0045,0053,0062,...,3465,3471,3475,3494,3498,3500,3502,3504,3514,Data analysis
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2136,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df2 = df
apriori_t = apriori(df2, min_support = 0.01, use_colnames = True, verbose = 1)
apriori_t

c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Processing 155 combinations | Sampling itemset size 54


,support,itemsets
0,0.011225,( 0014)
1,0.011225,( 0236)
2,0.014032,( 0257)
3,0.012629,( 0272)
4,0.013096,( 0364)
...,...,...
189,0.013096,"(2842, 2993, 3108, 3469)"
190,0.013096,"( 3503, 2993, 3108, 2842)"
191,0.013096,"( 3469, 2842, 2993, 3503)"
192,0.013096,"(2842, 3108, 3503, 3469)"


In [ ]:
# Let's view our interpretation values using the Associan rule function.
df_ar = association_rules(apriori_t, metric = "confidence", min_threshold = 0.6)
df_ar[:30]

In [11]:
result = course.loc[course['Course_id'] == "0992"]['Course Name'].iloc[0]
print('antecedents:', result)

antecedents: neural network deep learning


In [ ]:
for n, antecedents in enumerate(df_ar['antecedents']):
    if isinstance(antecedents, frozenset):
        antecedents = list(antecedents)
    antecedent_ids = ','.join(map(str, antecedents)).strip()  # Chuyển thành danh sách và tách các ID

    antecedent_names = []
    for antecedent_id in antecedent_ids.split(','):
        antecedent_id = antecedent_id.strip()
        result = course.loc[course['Course_id'] == antecedent_id]['Course Name'].iloc[0]
        antecedent_names.append(result)

    antecedent_names_str = ', '.join(antecedent_names)

    print(n)
    print('antecedents:', antecedent_names_str)

    consequents = df_ar['consequents'][n]
    if isinstance(consequents, frozenset):
        consequents = list(consequents)
    consequent_ids = ','.join(map(str, consequents)).strip()  # Chuyển thành danh sách và tách các ID

    consequent_names = []
    for consequent_id in consequent_ids.split(','):
        consequent_id = consequent_id.strip()
        result = course.loc[course['Course_id'] == consequent_id]['Course Name'].iloc[0]
        consequent_names.append(result)

    consequent_names_str = ', '.join(consequent_names)


In [13]:
def recommend_courses(enrolled_courses, df_ar, num_recommendations=5):
    # Tạo một danh sách để lưu trữ các khoá học được đề xuất
    recommended_courses = []

    # Duyệt qua từng tập luật kết hợp trong df_ar
    for index, row in df_ar.iterrows():
        antecedents = row['antecedents']
        consequents = row['consequents']

        # Chuyển các ID trong antecedents thành các ID không có khoảng cách
        antecedents_cleaned = [course_id.replace(" ", "") for course_id in antecedents]

        # Kiểm tra nếu có ít nhất một khoá học từ antecedents có trong danh sách enrolled_courses
        if any(course_id in antecedents_cleaned for course_id in enrolled_courses):
            # Lấy danh sách các khoá học trong consequents
            recommended_courses.extend(consequents)

    # Loại bỏ các khoá học đã đăng ký và lặp lại
    recommended_courses = list(set(recommended_courses) - set(enrolled_courses))

    # Chọn một số lượng giới hạn của khoá học để đề xuất
    if len(recommended_courses) > num_recommendations:
        recommended_courses = recommended_courses[:num_recommendations]

    return recommended_courses

In [28]:
enrolled_courses = ["0966", "1677", "1109"]
# ['1801', "0001","3356"]


# Đề xuất các khoá học dựa trên danh sách đã đăng ký và bảng df_ar
recommended_courses = recommend_courses(enrolled_courses, df_ar)
course_dict = dict(zip(course['Course_id'], course['Course Name']))
result =[]
for course_id in enrolled_courses:
    clean_course_id = course_id.replace(" ", "")  # Loại bỏ dấu cách
    course_name = course_dict.get(clean_course_id, 'Not Found') 
for course_id in recommended_courses:
    clean_course_id = course_id.replace(" ", "")  # Loại bỏ dấu cách
    course_name = course_dict.get(clean_course_id, 'Not Found')  # Lấy tên khoá học từ từ điển course_dict
    
    # Kiểm tra xem khoá học đã được đăng ký hay chưa
    if clean_course_id not in enrolled_courses:
        result.append(clean_course_id)
result

['0685', '0819', '2924']

In [15]:
def zip4id(id):
    stringid = str(id)
    
    if len(stringid) < 4:
        stringid = '0' * (4 - len(stringid)) + stringid
    
    return stringid
zip4id(999)

'0999'

In [16]:
print("Enrolled Courses:")
for course_id in enrolled_courses:
    clean_course_id = course_id.replace(" ", "")  # Loại bỏ dấu cách
    course_name = course_dict.get(clean_course_id, 'Not Found')  # Lấy tên khoá học từ từ điển course_dict
    print(f"- {course_name} (ID: {clean_course_id})")

Enrolled Courses:
- interactive word embedding use word vec plotly (ID: 0966)
- nlp twitter sentiment analysis (ID: 1677)
- sentiment analysis deep learning use bert (ID: 1109)


In [17]:
print("Recommend Courses:")
for course_id in recommended_courses:
    clean_course_id = course_id.replace(" ", "")  # Loại bỏ dấu cách
    course_name = course_dict.get(clean_course_id, 'Not Found')  # Lấy tên khoá học từ từ điển course_dict
    
    # Kiểm tra xem khoá học đã được đăng ký hay chưa
    if clean_course_id not in enrolled_courses:
        print(f"- {course_name} (ID: {clean_course_id})")

Recommend Courses:
- transfer learn nlp tensorflow hub (ID: 2497)


In [2]:
!pip install -q transformers
!pip install -q datasets tqdm pandas
!pip install -q sentencepiece

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a v

# Grammar Error Corection

In [3]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
# model_name = 'deep-learning-analytics/GrammarCorrector'
model_name = 't5_gec_model_03/t5_gec_model_03' # model path
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

# def correct_grammar(input_text,num_return_sequences):
#   batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
#   translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
#   tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
#   return tgt_text

In [4]:
def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [3]:
def mark_text(input_text, corrected_text):
    # Tách từng từ trong hai câu
    input_words = input_text.split()
    corrected_words = corrected_text.split()
    
    result = []

    # Duyệt qua từng từ trong hai câu
    for input_word, corrected_word in zip(input_words, corrected_words):
        # So sánh từ hiện tại trong hai câu
        if input_word == corrected_word:
            # Nếu từ giống nhau, thêm vào chuỗi kết quả với thẻ <span> bình thường
            result.append(f"<span>{input_word}</span>")
        else:
            # Nếu từ khác nhau, thêm vào chuỗi kết quả với thẻ <span> màu đỏ
            result.append(f"<span style='color: red;'>{corrected_word}</span>")

    # Kết hợp chuỗi kết quả thành một chuỗi duy nhất, thêm khoảng trắng giữa các thẻ <span>
    marked_text = ' '.join(result)

    return marked_text

corrected_text = "he is a teacher"
input_text = "he is an teacher"

marked_result = mark_text(input_text, corrected_text)
print(marked_result)

<span>he</span> <span>is</span> <span style='color: red;'>a</span> <span>teacher</span>


In [5]:
input_text = "summary: I am enjoys, writtings Articles ons AI and I also enjoyed write articling on AI."
num_return_sequences = 3 # Số lượng câu generate
corrected_text = correct_grammar(input_text, num_return_sequences)
corrected_text

['summary: I enjoy writing articles on AI and I also enjoy writing articles on AI.',
 'Summary: I enjoy writing articles on AI and I also enjoy writing articles on AI.',
 'summary: I enjoy writing articles on AI and I also enjoyed writing articles on AI.']

In [6]:
corrected_text

['summary: I enjoy writing articles on AI and I also enjoy writing articles on AI.',
 'Summary: I enjoy writing articles on AI and I also enjoy writing articles on AI.',
 'summary: I enjoy writing articles on AI and I also enjoyed writing articles on AI.']

# Question Generation and Fact - Check

In [7]:
# import fitz  # PyMuPDF
from transformers import T5ForConditionalGeneration, T5Tokenizer
import json

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [8]:
import re
import textract

# Đọc nội dung của tài liệu PDF
text = textract.process('Report.pdf', encoding='utf-8')

# Sử dụng biểu thức chính quy để cắt thành các đoạn văn
all_paragraphs = re.split(r'\s{2,}', text.decode('utf-8'))
num_paragraph= len(text)
print(num_paragraph)

13876


In [9]:
list_para = []
list_para = [para for para in all_paragraphs if len(para.split()) >= 20] # list of paragraphs which have more than 20 words
len(list_para)

21

In [10]:
# Summarize paragraph using t5 model

summaries = []
for i,paragraph in enumerate(list_para):
    input_text = "summarize: " + paragraph
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)
    
    # if i==3: break

In [11]:
list_para[:4]

['Abstract. This paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters.',
 "It has always been a tough choice for the students to choose the courses in different semesters in which there is possibility to score good grades apart from the interest in the course. IIIT-Delhi offers variety of courses with mandatory courses in first 4 semesters (with exception of 2 to 3 electives) and all elective courses from fifth semester onwards. Hence, choosing the courses based on the verbal recommendation from the seniors, instructors and fellowmates becomes a hectic task. For easing this process of course recommendation for an upcoming semester, we have developed a system which deploys simple yet powerful recommendation techniques such as auto-encoders, hybrid matrix factorization and similarity based approaches. It is a GUI based system which takes an input of student's ID (which is stored in the backend d

In [12]:
# merge into result 

result = [{"paragraph": list_para[i], "summary": summaries[i]} for i in range(len(summaries))]

In [13]:
result[:5]

[{'paragraph': 'Abstract. This paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters.',
  'summary': 'this paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters. this paper aims to develop a system that will help in recommendation of courses based on the performance of previous semesters.'},
 {'paragraph': "It has always been a tough choice for the students to choose the courses in different semesters in which there is possibility to score good grades apart from the interest in the course. IIIT-Delhi offers variety of courses with mandatory courses in first 4 semesters (with exception of 2 to 3 electives) and all elective courses from fifth semester onwards. Hence, choosing the courses based on the verbal recommendation from the seniors, instructors and fellowmates becomes a hectic task. For easing this

In [14]:
# Clone this responsitory for questions generator
# %cd question_generator
# !git clone https://github.com/amontgomerie/question_generator!git clone https://github.com/amontgomerie/question_generator
!pip install -r requirements.txt -qq
# !python run_qg.py --text_file question_generator/articles/twitter_hack.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [16]:
# import file from another folder
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'question_generator')

from questiongenerator import QuestionGenerator
qg = QuestionGenerator()


In [17]:
text = result[2]["summary"]
q  = qg.generate(text, num_questions=3)

Generating questions...



c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Evaluating QA pairs...



In [18]:
q

[{'question': 'how many students have taken the course?',
  'answer': 'the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.'},
 {'question': 'how many students have taken the iitt?',
  'answer': 'the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches.'},
 {'question': 'how many subjects do students have taken?',
  'answer': [{'answer': '7 Computer Science', 'correct': False},
   {'answer': '739', 'correct': False},
   {'answer': '306', 'correct': True}]}]

In [19]:
print(text)
for i in range(len(q)): print(q[i]["question"]) #  7 Computer Science passout batches.

the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches. the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.
how many students have taken the course?
how many students have taken the iitt?
how many subjects do students have taken?


### Fact -check

In [20]:
import torch
import torch.nn as nn

from transformers import BertTokenizer, BertModel

In [27]:
class BERTClassificationModel(nn.Module):
    def __init__(self, bert_model_name, num_labels):
        super(BERTClassificationModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size * 2, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = torch.cat((outputs.last_hidden_state[:, 0, :], outputs.last_hidden_state[:, -1, :]), dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [28]:
device = torch.device("cpu")


In [33]:
import torch
model_path = "bert_classification_model.pth"
# Tạo mô hình mới
loaded_model = BERTClassificationModel('bert-base-uncased', num_labels=3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
loaded_tokenizer = BertTokenizer.from_pretrained("tokenizer")

In [40]:
if torch.cuda.is_available():
    loaded_model.load_state_dict(torch.load(model_path))
else:
    loaded_model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')), strict=False )

In [41]:
max_seq_length = 128
def predict_premise_hypothesis(premise_text, hypothesis_text, model, tokenizer):
    # Chuẩn bị dữ liệu đầu vào cho mô hình
    inputs = tokenizer(premise_text, hypothesis_text, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Dự đoán
    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probabilities = torch.softmax(logits, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item()

    return predicted_label

pre = ["entailment", "neutral", "contradiction"]
# Sử dụng hàm predict_premise_hypothesis để dự đoán
premise_text = "The likelihood is 100%"
hypothesis_text = "Maybe the probability is 100%"
predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)

print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])


premise_text : The likelihood is 100% 
 hypothesis_text : Maybe the probability is 100%
Predicted Label: entailment


In [42]:
premise_text = "The dataset had acquired for the student of 7 computer science"
hypothesis_text = result[2]["summary"]

predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)
print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])

premise_text : The dataset had acquired for the student of 7 computer science 
 hypothesis_text : the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches. the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.
Predicted Label: entailment


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import csv

# Mở tệp CSV để đọc
with open('mdl_assign.csv', newline='') as csvfile:
    # Tạo đối tượng đọc CSV
    csvreader = csv.reader(csvfile)
    
    # Đọc tiêu đề từ dòng đầu tiên của tệp CSV
    headers = next(csvreader)
    
for header in headers:
    globals()[header] = 1     # Nếu bạn muốn thêm vào danh sách, hãy sử dụng:
  
print(id)      # header_list.append(header)

1


In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, jsonify
from flask_sqlalchemy import SQLAlchemy
import pickle
import csv

loaded_model = pickle.load(open('trained_model.sav', 'rb'))

vector = pickle.load(open('vector_conv.sav', 'rb'))

app = Flask(__name__)
app.secret_key = "Secret Key"

app.config["SQLALCHEMY_DATABASE_URI"] = 'mysql://root:''@localhost/moodle'
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

db = SQLAlchemy(app)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_3508/82547003.py:6: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  loaded_model = pickle.load(open('trained_model.sav', 'rb'))
c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/st

In [ ]:
headers

['name',
 'intro',
 'introformat',
 'alwaysshowdescription',
 'nosubmissions',
 'submissiondrafts',
 'sendnotifications',
 'sendlatenotifications',
 'duedate',
 'allowsubmissionsfromdate',
 'grade',
 'timemodified',
 'requiresubmissionstatement',
 'completionsubmit',
 'cutoffdate',
 'gradingduedate',
 'teamsubmission',
 'requireallteammemberssubmit',
 'teamsubmissiongroupingid',
 'blindmarking',
 'hidegrader',
 'revealidentities',
 'attemptreopenmethod',
 'maxattempts',
 'markingworkflow',
 'markingallocation',
 'sendstudentnotifications',
 'preventsubmissionnotingroup',
 'activity',
 'activityformat',
 'timelimit',
 'submissionattachments']

In [ ]:
class Mdl_assign(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    course = db.Column(db.String(100))
    name = db.Column(db.String(100))
    intro = db.Column(db.String(100))
    introformat = db.Column(db.String(100))
    alwaysshowdescription = db.Column(db.String(100))
    nosubmissions = db.Column(db.String(100))
    submissiondrafts = db.Column(db.String(100))
    sendnotifications = db.Column(db.String(100))
    sendlatenotifications = db.Column(db.String(100))
    duedate = db.Column(db.String(100))
    allowsubmissionsfromdate = db.Column(db.String(100))
    grade = db.Column(db.String(100))
    timemodified = db.Column(db.String(100))
    requiresubmissionstatement = db.Column(db.String(100))
    completionsubmit = db.Column(db.String(100))
    cutoffdate = db.Column(db.String(100))
    gradingduedate = db.Column(db.String(100))
    teamsubmission = db.Column(db.String(100))
    requireallteammemberssubmit = db.Column(db.String(100))
    teamsubmissiongroupingid = db.Column(db.String(100))
    blindmarking = db.Column(db.String(100))
    hidegrader = db.Column(db.String(100))
    revealidentities = db.Column(db.String(100))
    attemptreopenmethod = db.Column(db.String(100))
    maxattempts = db.Column(db.String(100))
    markingworkflow = db.Column(db.String(100))
    markingallocation = db.Column(db.String(100))
    sendstudentnotifications = db.Column(db.String(100))
    preventsubmissionnotingroup = db.Column(db.String(100))
    activity = db.Column(db.String(100))
    activityformat = db.Column(db.String(100))
    timelimit = db.Column(db.String(100))
    submissionattachments = db.Column(db.String(100))
    def __init__(self, id, course, name, intro, introformat, alwaysshowdescription, nosubmissions, submissiondrafts, 
    sendnotifications, sendlatenotifications, duedate, allowsubmissionsfromdate, grade, timemodified,requiresubmissionstatement, completionsubmit, cutoffdate,
    gradingduedate, teamsubmission, requireallteammemberssubmit, teamsubmissiongroupingid, blindmarking, hidegrader, revealidentities, attemptreopenmethod, maxattempts,
    markingworkflow, markingallocation, sendstudentnotifications, preventsubmissionnotingroup, activity, activityformat, timelimit, submissionattachments):
        self.id = id
        self.course = course
        self.name = name
        self.intro = intro
        self.introformat = introformat
        self.alwaysshowdescription = alwaysshowdescription
        self.nosubmissions = nosubmissions
        self.submissiondrafts = submissiondrafts
        self.sendnotifications = sendnotifications
        self.sendlatenotifications = sendlatenotifications
        self.duedate = duedate
        self.allowsubmissionsfromdate = allowsubmissionsfromdate
        self.grade = grade
        self.timemodified = timemodified
        self.requiresubmissionstatement = requiresubmissionstatement
        self.completionsubmit = completionsubmit
        self.cutoffdate = cutoffdate
        self.gradingduedate = gradingduedate
        self.teamsubmission = teamsubmission
        self.requireallteammemberssubmit = requireallteammemberssubmit
        self.teamsubmissiongroupingid = teamsubmissiongroupingid
        self.blindmarking = blindmarking
        self.hidegrader = hidegrader
        self.revealidentities = revealidentities
        self.attemptreopenmethod = attemptreopenmethod
        self.maxattempts = maxattempts
        self.markingworkflow = markingworkflow
        self.markingallocation = markingallocation
        self.sendstudentnotifications = sendstudentnotifications
        self.preventsubmissionnotingroup = preventsubmissionnotingroup
        self.activity = activity
        self.activityformat = activityformat
        self.timelimit = timelimit
        self.submissionattachments = submissionattachments